In [184]:
#Get the libraries

import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt

In [185]:
#Get the data
churn_df = pd.read_csv("churn.csv")
churn_df = churn_df.drop(columns="PaymentMethod")
churn_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,70.70,151.65,Yes


In [186]:
#Remove customer ID
churn_df = churn_df.loc[:, 'gender':'Churn']
churn_df.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 19 columns):
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), object(16)
memory usage: 1.0+ MB


In [187]:
#Replace " " with nan and drop none values
churn_df['TotalCharges'].replace(' ', np.nan, inplace=True)
churn_df.dropna(subset=['TotalCharges'], inplace=True)

churn_df.head(2)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,56.95,1889.5,No


In [188]:
#Change Total Charges to Numeric Values
churn_df["TotalCharges"] = pd.to_numeric(churn_df["TotalCharges"])

In [189]:
#Preprocessing
X = churn_df.loc[:, 'gender':'Churn'].values
X[0:5]

array([['Female', 0, 'Yes', 'No', 1, 'No', 'No phone service', 'DSL',
        'No', 'Yes', 'No', 'No', 'No', 'No', 'Month-to-month', 'Yes',
        29.85, 29.85, 'No'],
       ['Male', 0, 'No', 'No', 34, 'Yes', 'No', 'DSL', 'Yes', 'No',
        'Yes', 'No', 'No', 'No', 'One year', 'No', 56.95, 1889.5, 'No'],
       ['Male', 0, 'No', 'No', 2, 'Yes', 'No', 'DSL', 'Yes', 'Yes', 'No',
        'No', 'No', 'No', 'Month-to-month', 'Yes', 53.85, 108.15, 'Yes'],
       ['Male', 0, 'No', 'No', 45, 'No', 'No phone service', 'DSL',
        'Yes', 'No', 'Yes', 'Yes', 'No', 'No', 'One year', 'No', 42.3,
        1840.75, 'No'],
       ['Female', 0, 'No', 'No', 2, 'Yes', 'No', 'Fiber optic', 'No',
        'No', 'No', 'No', 'No', 'No', 'Month-to-month', 'Yes', 70.7,
        151.65, 'Yes']], dtype=object)

In [190]:
from sklearn import preprocessing

_gender = preprocessing.LabelEncoder()
_gender.fit(['Female','Male'])
X[:,0] = _gender.transform(X[:,0]) 

#Partner
_partner = preprocessing.LabelEncoder()
_partner.fit(['Yes', 'No'])
X[:,2] = _partner.transform(X[:,2])

#Dependents
_dependents = preprocessing.LabelEncoder()
_dependents.fit(['Yes', 'No'])
X[:,3] = _dependents.transform(X[:,3])

#PhoneService
_pservice = preprocessing.LabelEncoder()
_pservice.fit(['Yes', 'No'])
X[:,5] = _pservice.transform(X[:,5]) 

#Multilines
_multilines = preprocessing.LabelEncoder()
_multilines.fit(['Yes', 'No', 'No phone service'])
X[:,6] = _multilines.transform(X[:,6]) 

#Internet service
_is = preprocessing.LabelEncoder()
_is.fit(['DSL', 'Fiber optic', 'No'])
X[:,7] = _is.transform(X[:,7]) 

#Online security
_onlinesec = preprocessing.LabelEncoder()
_onlinesec.fit(['Yes', 'No', 'No internet service'])
X[:,8] = _onlinesec.transform(X[:,8])

#Online Backup
_onlinebac = preprocessing.LabelEncoder()
_onlinebac.fit(['Yes', 'No', 'No internet service'])
X[:,9] = _onlinebac.transform(X[:,9]) 

#Device protection
_dp = preprocessing.LabelEncoder()
_dp.fit(['Yes', 'No', 'No internet service'])
X[:,10] = _dp.transform(X[:,10])

#Tech support
_ts = preprocessing.LabelEncoder()
_ts.fit(['Yes', 'No', 'No internet service'])
X[:,11] = _ts.transform(X[:,11]) 

#StreamingTv
_st = preprocessing.LabelEncoder()
_st.fit(['Yes', 'No', 'No internet service'])
X[:,12] = _st.transform(X[:,12])

#StreamingMovies
_sm = preprocessing.LabelEncoder()
_sm.fit(['Yes', 'No', 'No internet service'])
X[:,13] = _sm.transform(X[:,13])

#Contract
_contract = preprocessing.LabelEncoder()
_contract.fit(['Month-to-month', 'One year', 'Two year'])
X[:,14] = _contract.transform(X[:,14])

#Paperless billing
_pb = preprocessing.LabelEncoder()
_pb.fit(['Yes','No'])
X[:,15] = _pb.transform(X[:,15])

X[0:1]

array([[0, 0, 1, 0, 1, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 1, 29.85, 29.85,
        'No']], dtype=object)